# **SVD Comparisons (new C++20 library vs. numpy.linalg.svd)**

## Team H
* Evan Ram
* Prateek Makhija
* James Douthit
* Garrett Hempy

## Introduction / methods

// todo

## Dependencies

We use a new library called Matrix written in C++20 (the C++20 language version should be finalized later this month). It is written by Feng Wang and can be found [here](https://github.com/fengwang/matrix). The library is distributed as a huge single header file (see `./fengwang-matrix/matrix.hpp`).

We also depend on `numpy` for matrix stuff in Python and `pillow` (Python image processing library) for loading in the C++ library's exported matrices (which can be serialized to bitmap image files).

In [1]:
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pillow

/bin/sh: {sys.executable}: command not found
/bin/sh: {sys.executable}: command not found


Feng Wang's Matrix library is difficult to compile on non-linux machines because of the new C++20 supporting compilers, so we require [Docker](https://www.docker.com/) to be installed in order to compile and run our driver code.

The below `FengWangSVD` class will run the Docker commands for us so we can test its SVD implementation directly from Python.

In [2]:
import sys
import numpy as np
import os
import subprocess
from PIL import Image
import datetime

class FengWangSVD:

    def __init__(self, A):
        """
        `A` should be a 2d numpy array
        """
        
        self._A = A
    
    @staticmethod
    def build():
        """
        Builds Feng Wang's Matrix library SVD tester using Docker.

        New C++20 features are not available on many machines,
        so please have Docker installed.
        """

        # Use bang command in Jupyter notebook since we don't care about this command's output
        !docker build -t fengwang-matrix-svd .
        
        # Technically we don't run any Python in this method
        pass

    def run(self):
        """
        Runs the C++ test program for the fengwang/matrix library.
        Returns its command line output as a list of lines from stdout.

        Not using bang command b/c we want to process the output to get timings.
        Timing from the start of this method to the end of it is pointless since
        it includes overhead of process creation.
        """

        self._write_matrix()
        cmd = f'docker run -v {self._program_io_path()}:/program_io fengwang-matrix-svd'.split()
        proc_out = subprocess.check_output(cmd)
        lines = proc_out.decode('utf-8').split('\n')
        self._read_stats(lines)
        self._read_matrices()
    
    @property
    def stats(self):
        """
        A dict containing the stat results from running the program.
        """

        if not hasattr(self, '_stats'):
            raise Exception('Please call run() first')

        return self._stats
    
    def _program_io_path(self):
        path = os.path.join(os.getcwd(), 'fengwang-matrix/program_io')
        if not os.path.exists(path):
            os.makedirs(path)
        return path
    
    def _write_matrix(self):
        """
        Write matrix A as input data to the program.
        """
        
        path = os.path.join(self._program_io_path(), 'input.npy')
        with open(path, 'wb') as f:
            np.save(f, self._A)
    
    def _read_stats(self, stdout_lines):
        """
        Parse stdout for key/value pairs and assign it to self._stats
        """
        
        stats = {}
        start_processing = False
        
        for i, line in enumerate(stdout_lines):
            # Most lines end with carriage return '\r' for some reason
            line = line.strip()
            
            if line == '!!BEGIN-STATS!!':
                # All stdout lines after this magic string will be parsed as K/V pairs
                start_processing = True
                continue
            elif not start_processing:
                # Program spits out diagnostic data on `load_npy`, cant seem to disable it...
                continue
    
            if len(line) == 0:
                # Blank line
                continue
        
            [k, v] = line.split(':=', 1)
            k = k.lower().strip()
            v = v.strip()
            
            # Output numbers should all be integers, to avoid differences in
            # floating point arithmetic between Python and C++ tests
            if v.isdigit():
                v = int(v)
                
            stats[k] = v
            
        self._stats = stats
        
    def _read_matrices(self):
        """
        Read in the matrices from the generated .bmp files the program created.
        Will populate self.{U, S, V, A_prime}
        """
        
        matrices = ['U', 'S', 'V', 'A_prime']
        
        for m in matrices:
            path = os.path.join(self._program_io_path(), m + '.bmp')
            img = Image.open(path).convert('L') # 'L' for grayscale ... weird const but ok
            
            # Rescale b/c the .bmp format maps 0.0-1.0 onto integers 0-255
            setattr(self, m + '_bmp', np.array(img) / 255)
        
        for m in matrices:
            path = os.path.join(self._program_io_path(), m + '.txt')
            with open(path, 'r') as myfile:
                data = myfile.read()
                setattr(self, m, self._txt_to_mat(data))
    
    def _txt_to_mat(self, data):
        
        lines = data.split('\n')
        A = []
        
        for line in lines:
            A.append([float(x) for x in line.split('\t')[:-1]])
        
        return np.array(A[:-1])

Run the next cell to build the C++ driver code that builds our driver executable for Feng Wang's Matrix SVD function.

In [3]:
FengWangSVD.build()

Sending build context to Docker daemon  797.7kB

Step 1/5 : FROM gcc:latest
 ---> 2f9778ee181e
Step 2/5 : COPY ./fengwang-matrix /app
 ---> e4b9e56f436a
Step 3/5 : WORKDIR /app
 ---> Running in db3cf072676c
Removing intermediate container db3cf072676c
 ---> 338d6d1d49bc
Step 4/5 : RUN make
 ---> Running in 3507356a166e
g++ -std=c++2a -Wall -Wextra -O2 -pthread -o svdimage main.cpp -lstdc++fs
Removing intermediate container 3507356a166e
 ---> 0693761823d7
Step 5/5 : CMD ["./svdimage"]
 ---> Running in 04b7d51a6446
Removing intermediate container 04b7d51a6446
 ---> a9fe4351002d
Successfully built a9fe4351002d
Successfully tagged fengwang-matrix-svd:latest
SECURITY WARNING: You are building a Docker image from Windows against a non-Windows Docker host. All files and directories added to build context will have '-rwxr-xr-x' permissions. It is recommended to double check and reset permissions for sensitive files and directories.


Let's just see it working real quick.

In [4]:
A = np.random.rand(100, 100) * 1000
fw_svd = FengWangSVD(A)
fw_svd.run()

print(f'Time to run SVD algo: {fw_svd.stats["performance-svd-us"]}μs')
print(f'\nTime to construct A\' (U x S x V^T): {fw_svd.stats["performance-aprime-us"]}μs')

# Fairly large residuals (numerical error in .bmp format limitations)
print('\nResiduals between A and the product U x S x V^T')
print(np.subtract(A, fw_svd.U @ fw_svd.S @ fw_svd.V.T))

# Smaller residuals
print('\nResiduals between A and A\' (reconstructed matrix from Feng Wang\'s SVD)')
print(np.subtract(A, fw_svd.A_prime))
      
print('\n\nThe .bmp file export however results in crazy high residuals and might be broken:')

print('\n(bmp) Residuals between A and the product U x S x V^T')
print(np.subtract(A, fw_svd.U_bmp @ fw_svd.S_bmp @ fw_svd.V_bmp.T))
      
print('\n(bmp) Residuals between A and A\' (reconstructed matrix from Feng Wang\'s SVD)')
print(np.subtract(A, fw_svd.A_prime_bmp))

Time to run SVD algo: 17477μs

Time to construct A' (U x S x V^T): 3193μs

Residuals between A and the product U x S x V^T
[[ 1.30739863e-12  5.68434189e-13  3.89377419e-12 ... -6.25277607e-13
   4.54747351e-13  1.59161573e-12]
 [ 3.41060513e-13  2.84217094e-12  1.02318154e-12 ... -2.27373675e-13
   6.25277607e-13  1.16706644e-12]
 [ 6.53699317e-13  1.25055521e-12 -2.27373675e-13 ... -7.10542736e-14
   1.36424205e-12 -3.12638804e-13]
 ...
 [ 1.94955163e-12  3.18323146e-12 -7.95807864e-13 ...  2.27373675e-12
  -1.13686838e-13 -6.82121026e-13]
 [ 2.54374299e-12  2.84217094e-12  3.83693077e-13 ...  1.13686838e-12
  -1.70530257e-12 -5.11590770e-13]
 [ 3.12638804e-13  5.40012479e-12  2.38742359e-12 ... -3.41060513e-13
   1.25055521e-12 -3.41060513e-13]]

Residuals between A and A' (reconstructed matrix from Feng Wang's SVD)
[[ 1.08002496e-12  7.95807864e-13  3.89377419e-12 ... -2.84217094e-13
   5.68434189e-13  1.47792889e-12]
 [ 3.26849658e-13  2.95585778e-12  7.95807864e-13 ... -4.5474735

## Breaking it

//todo
The product of the {U,S,V}.bmp exported matrices loses a lot of precision unless we get the UxSxV^T product before dumping to bmp (A_prime.bmp). Also note the bug where an integer matrix crashes the library's loader code

## Comparison with np.linalg.svd

We will just ignore the broken .bmp format for Feng Wang in our comparison because either we are using it wrong or the exported data is highly different for some reason.

In [17]:
begin = datetime.datetime.now()
np_U, np_S, np_Vt = np.linalg.svd(A)
end = datetime.datetime.now()
elapsed_np_svd = end - begin
micro_np_svd = elapsed_np_svd.total_seconds() * 1000000
np_S = np.diag(np_S) # Why is np_S not already a diagonal matrix before this?
b = datetime.datetime.now()
np_A_prime = np_U @ np_S @ np_Vt
e = datetime.datetime.now()
elapsed_np_a_prime = e - b
micro_np_a_prime = elapsed_np_a_prime.total_seconds() * 1000000


print('\nResiduals between A and A\' (reconstructed matrix from numpy\'s SVD)')
print(np.subtract(A, np_A_prime))

print('\nResiduals between Feng Wang\'s U, S, and V factors:')
print('\nU diff: ', np.subtract(np_U, fw_svd.U))
print('\nS diff: ', np.subtract(np_S, fw_svd.S))
print('\nV diff: ', np.subtract(np_Vt.T, fw_svd.V))

print('\nResiduals between Feng Wang\'s A\' and numpy\'s A\' (reconstruction of A):')
print('A\' diff: ')
print(abs(np_A_prime - fw_svd.A_prime))
print("\nPerformance numpy SVD:",micro_np_svd,"microseconds")
print("\nTime to construct numpy A prime:",micro_np_a_prime,"microseconds")


Residuals between A and A' (reconstructed matrix from numpy's SVD)
[[-2.89901436e-12  2.27373675e-13  2.13162821e-12 ...  2.04636308e-12
   3.63797881e-12  2.52953214e-12]
 [-3.19744231e-12 -1.93267624e-12  3.18323146e-12 ...  3.41060513e-13
   0.00000000e+00  3.44613227e-13]
 [-2.41584530e-12 -2.27373675e-13 -3.06954462e-12 ...  1.83320026e-12
   6.82121026e-13 -1.73372428e-12]
 ...
 [-2.48778775e-12 -2.61479727e-12  5.68434189e-13 ... -6.82121026e-13
   1.13686838e-13  5.68434189e-13]
 [-3.06954462e-12  2.50111043e-12 -3.41060513e-13 ... -1.70530257e-12
  -6.82121026e-13  7.67386155e-13]
 [-2.27373675e-12 -1.93267624e-12  1.70530257e-12 ... -2.27373675e-13
   1.47792889e-12  2.27373675e-13]]

Residuals between Feng Wang's U, S, and V factors:

U diff:  [[ 2.08166817e-16 -7.77156117e-16 -3.79600865e-15 ...  5.85989590e-15
  -3.06340923e-01  3.06340923e-01]
 [-1.38777878e-16 -2.47024623e-15 -3.22658567e-15 ...  3.05311332e-16
   1.74235039e-01 -1.74235039e-01]
 [ 0.00000000e+00 -2.945

## Results and interpretation

When we started to print our results, we noticed that the residuals were extremely high. We fixed this by switching the format of the file we exported our data to from a bmp to txt. The bmp conversion either has a bug, or was losing a lot of precision with our data because it caused our residuals to shoot up to the thousands. Both implementations of SVD are similar as the resulting data has very small differences. We also ran into the issue that numpy does not automatically convert sigma to a diagonal matrix, so we had to specifically set sigma to diagonal after we got the matrix from numpy.linalg.svd.



James TODO: Discuss our results in more of a math perspective and interpret them? Lol im sure you are a lot more educated on this subject than we are. Add some more conclusions and open questions? All we really need is to beef up what we say in our results and conclusion.

## Conclusions and open questions

    Overall we got it to work. At the beginning we had issues with the residuals being high, but after figuring out how to fix it, we were able to get the results we were looking for. 
    
    Our initial prediction is that the numpy implementation of SVD will be more optimized than fengwang's implementation in C++, due to the fact that numpy is one of the main libraries used for matrix manipulations in Python. After running performance tests, the numpy SVD implementation performed much faster than the c++ implementation; by a factor of almost 4. When reconstructing A prime as well, we notice that numpy implementation is about 3 times faster than the c++ implementation. These reults match up with our intial prediction that the numpy implementation would out perform the c++ version.
    After doing more research, we find out that actually c++ code is not neccessarly slower than the python code. What Feng Wang most likely forgot to do is optimize the compilation with gcc in his C++ code. 
    
Sources:

https://stackoverflow.com/questions/41365723/why-my-python-numpy-code-is-faster-than-c
https://github.com/fengwang/matrix
https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html 

Questions: 
1. Why does the bitmap function not work? Was it our error or an error with fengwang Matrix?

2. Why was the numpy implementation not convert sigma to a diagonal matrix?